# はじめに
このノートブックでは、拡張固有表現認識のベースラインモデルの作成を行います。
まずはデータセットを読み込み、整形します。
その後、ベースラインモデルを構築し、評価を行います。

## データセットの読み込み
この節では、拡張固有表現のデータセットを読み込みます。
データセットには、毎日新聞1995に対して拡張固有表現が付与されたデータセットを用います。
以下のコードを実行して、文字ベースのIOB2形式で読み込みます。

In [1]:
import os
import sys
sys.path.append('../')
from entitypedia.evaluation.converter import to_iob2

mainichi_dir = '../data/raw/corpora/mainichi'
X, y = to_iob2(mainichi_dir)
print(' '.join(X[0][:50]))
print(' '.join(y[0][:50]))

IndexError: list index out of range

上記に示したように読み込んだデータセットでは文字単位でラベルがついています。今回は単語単位で認識するベースラインモデルを作りたいため、単語レベルにラベルを付け直します。

タスクとしては以下の通りです。

* 文字のリストを結合して文字列にする
* 文字列を形態素解析器で解析し、分かち書きする
* 分かち書きした単語のリストに対してラベルを付け直す。

まずは文字のリストを結合して文字列にします。

In [10]:
docs = [''.join(doc) for doc in X]
docs[0][:100]

'\n\u3000◇国際貢献など４点、ビジョンの基本示す\n\u3000村山富市首相は年頭の記者会見で、「創造とやさしさの国造りのビジョン」と題する所感を発表した。今月中に首相を囲む学者グループが発表する「村山ビジョン」の基本'

次に、結合した文字列に対して形態素解析を行います。
形態素解析機にはMeCabを使用します。ついでに品詞情報も取得しておきましょう。

In [12]:
import MeCab
t = MeCab.Tagger()


def tokenize(sent):
    tokens = []
    t.parse('')  # for UnicodeDecodeError
    node = t.parseToNode(sent)

    while node:
        feature = node.feature.split(',')
        surface = node.surface    # 表層形
        pos = feature[0]          # 品詞
        tokens.append((surface, pos))
        node = node.next

    return tokens[1:-1]

tokenized_docs = [[d[0] for d in tokenize(doc)] for doc in docs]
poses = [[d[1] for d in tokenize(doc)] for doc in docs]

print(tokenized_docs[0][:10])
print(poses[0][:10])

['\u3000', '◇', '国際', '貢献', 'など', '４', '点', '、', 'ビジョン', 'の']
['記号', '記号', '名詞', '名詞', '助詞', '名詞', '名詞', '記号', '名詞', '助詞']


これで分かち書きまではできました。その後が若干面倒です。ラベルを単語単位で付け直す必要があります。
以下の手順でやってみましょう。

1. 形態素を1つ取り出す
2. 形態素を構成するラベルを文字列マッチングによって取り出す
3. ラベルを修正する

In [13]:
tags = []
for t_doc, doc, label in zip(tokenized_docs, docs, y):
    i = 0
    doc_tags = []
    for word in t_doc:
        j = len(word)
        while not doc[i:].startswith(word):  # correct
            i += 1
        tag = label[i: i+j][0]
        # print('{}\t{}'.format(word, tag))
        doc_tags.append(tag)
        i += j
    tags.append(doc_tags)
    # break

対応付けができているか確認してみましょう。

In [15]:
for word, tag in zip(tokenized_docs[0][:20], tags[0][:20]):
    print('{}\t{}'.format(word, tag))

　	O
◇	O
国際	O
貢献	O
など	O
４	O
点	O
、	O
ビジョン	O
の	O
基本	O
示す	O
　	O
村山	B-person
富市	I-person
首相	B-position_vocation
は	O
年頭	B-date
の	O
記者	B-position_vocation


大丈夫そうですね。では`tokenized_docs`と`tags`を`X`と`y`に代入してやりましょう。

In [16]:
X = tokenized_docs
y = tags

以上でデータの読み込みと整形は完了しました。
次はベースラインモデルを作成します。

# ベースラインモデルの作成
本節では拡張固有表現を認識するベースラインモデルを作成します。
現在の固有表現認識ではBi-LSTMとCRFを組み合わせたモデルがよく用いられます。しかし、今回のように認識するタグ数が多い場合、CRFを入れると計算量が非常に多くなり、現実的な時間で問題を解くことができなくなります。したがって、まずはシンプルなモデルで解いてみましょう。

ここでは、まず単純な単語ベースBi-LSTMを試してみます。計算時間が多いようだったら、更に簡単なモデルを検討します。

ではまずは、データセットを学習用と検証用に分割しましょう。

In [18]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

これでデータセットを分割できました。
現在、データセットの中は文字列で表現されています。これではモデルにデータを与えることができないので前処理を行います。
前処理のためのコードを定義していきましょう。
具体的な前処理としては、以下を行います。

* 単語を数字に変換
* 系列長の統一

少々長いですが以下のように定義できます。

In [25]:
import itertools
import re

import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.externals import joblib
from keras.preprocessing.sequence import pad_sequences

UNK = '<UNK>'
PAD = '<PAD>'

class Preprocessor(BaseEstimator, TransformerMixin):

    def __init__(self,
                 lowercase=True,
                 num_norm=True,
                 vocab_init=None,
                 padding=True,
                 return_lengths=True):

        self.lowercase = lowercase
        self.num_norm = num_norm
        self.padding = padding
        self.return_lengths = return_lengths
        self.vocab_word = None
        self.vocab_tag  = None
        self.vocab_init = vocab_init or {}

    def fit(self, X, y):
        words = {PAD: 0, UNK: 1}
        tags  = {PAD: 0}

        for w in set(itertools.chain(*X)) | set(self.vocab_init):
            if w not in words:
                words[w] = len(words)

        for t in itertools.chain(*y):
            if t not in tags:
                tags[t] = len(tags)

        self.vocab_word = words
        self.vocab_tag  = tags

        return self

    def transform(self, X, y=None):
        """transforms input(s)
        Args:
            X: list of list of words
            y: list of list of tags
        Returns:
            numpy array: sentences
            numpy array: tags
        Examples:
            >>> X = [['President', 'Obama', 'is', 'speaking']]
            >>> print(self.transform(X))
            [
                [1999, 1037, 22123, 48388],       # word ids
            ]
        """
        words = []
        lengths = []
        for sent in X:
            word_ids = []
            lengths.append(len(sent))
            for word in sent:
                word_ids.append(self.vocab_word.get(word, self.vocab_word[UNK]))

            words.append(word_ids)

        if y is not None:
            y = [[self.vocab_tag[t] for t in sent] for sent in y]

        if self.padding:
            maxlen = max(lengths)
            sents = pad_sequences(words, maxlen, padding='post')
            if y is not None:
                y = pad_sequences(y, maxlen, padding='post')
                y = dense_to_one_hot(y, len(self.vocab_tag), nlevels=2)

        else:
            sents = words

        if self.return_lengths:
            lengths = np.asarray(lengths, dtype=np.int32)
            lengths = lengths.reshape((lengths.shape[0], 1))
            sents = [sents, lengths]

        return (sents, y) if y is not None else sents

    def inverse_transform(self, y):
        indice_tag = {i: t for t, i in self.vocab_tag.items()}
        return [indice_tag[y_] for y_ in y]

    def vocab_size(self):
        return len(self.vocab_word)

    def tag_size(self):
        return len(self.vocab_tag)


def dense_to_one_hot(labels_dense, num_classes, nlevels=1):
    """Convert class labels from scalars to one-hot vectors."""
    if nlevels == 1:
        num_labels = labels_dense.shape[0]
        index_offset = np.arange(num_labels) * num_classes
        labels_one_hot = np.zeros((num_labels, num_classes), dtype=np.int32)
        labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
        return labels_one_hot
    elif nlevels == 2:
        # assume that labels_dense has same column length
        num_labels = labels_dense.shape[0]
        num_length = labels_dense.shape[1]
        labels_one_hot = np.zeros((num_labels, num_length, num_classes), dtype=np.int32)
        layer_idx = np.arange(num_labels).reshape(num_labels, 1)
        # this index selects each component separately
        component_idx = np.tile(np.arange(num_length), (num_labels, 1))
        # then we use `a` to select indices according to category label
        labels_one_hot[layer_idx, component_idx, labels_dense] = 1
        return labels_one_hot
    else:
        raise ValueError('nlevels can take 1 or 2, not take {}.'.format(nlevels))


def prepare_preprocessor(X, y, use_char=True):
    p = Preprocessor()
    p.fit(X, y)

    return p

p = prepare_preprocessor(X, y)

前処理の関数を定義できたので、次にデータ生成部分の処理を描いてあげます。
これは、バッチごとに前処理器を用いてデータを生成する処理になります。
以下のように定義できます。

In [35]:
def batch_iter(data, labels, batch_size, shuffle=False, preprocessor=None):
    num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1

    def data_generator():
        """
        Generates a batch iterator for a dataset.
        """
        data_size = len(data)
        while True:
            # Shuffle the data at each epoch
            if shuffle:
                shuffle_indices = np.random.permutation(np.arange(data_size))
                shuffled_data = data[shuffle_indices]
                shuffled_labels = labels[shuffle_indices]
            else:
                shuffled_data = data
                shuffled_labels = labels

            for batch_num in range(num_batches_per_epoch):
                start_index = batch_num * batch_size
                end_index = min((batch_num + 1) * batch_size, data_size)
                X, y = shuffled_data[start_index: end_index], shuffled_labels[start_index: end_index]
                if preprocessor:
                    yield preprocessor.transform(X, y)
                else:
                    yield X, y

    return num_batches_per_epoch, data_generator()


BATCH_SIZE = 32
train_steps, train_batches = batch_iter(
    x_train, y_train, BATCH_SIZE, preprocessor=p)
valid_steps, valid_batches = batch_iter(
    x_valid, y_valid, BATCH_SIZE, preprocessor=p)

ではモデルを定義しましょう。フレームワークにはKerasを使用します。

In [36]:
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Input, Dropout
from keras.models import Model


def build_model(vocab_size, ntags, embedding_size=100, n_lstm_units=100, dropout=0.5):
    sequence_lengths = Input(batch_shape=(None, 1), dtype='int32')
    word_ids = Input(batch_shape=(None, None), dtype='int32')
    word_embeddings = Embedding(input_dim=vocab_size,
                                output_dim=embedding_size,
                                mask_zero=True)(word_ids)
    x = Dropout(dropout)(word_embeddings)

    x = Bidirectional(LSTM(units=n_lstm_units, return_sequences=True))(x)
    x = Dropout(dropout)(x)
    x = Dense(n_lstm_units, activation='tanh')(x)
    pred = Dense(ntags, activation='softmax')(x)

    model = Model(inputs=[word_ids, sequence_lengths], outputs=[pred])

    return model

model = build_model(p.vocab_size(), p.tag_size())

以上で学習の準備が整いました。実際に学習させてみましょう。
最適化アルゴリズムには`Adam`を使用します。

In [38]:
from keras.optimizers import Adam

MAX_EPOCH = 5


model.compile(loss='categorical_crossentropy',
                          optimizer=Adam(),
                          metrics=['acc'],
                         )

model.fit_generator(generator=train_batches,
                    steps_per_epoch=train_steps,
                    validation_data=valid_batches,
                    validation_steps=valid_steps,
                    epochs=MAX_EPOCH)

Epoch 1/5
180/180 [==============================] - 2133s - loss: 0.8308 - acc: 0.8659 - val_loss: 0.7094 - val_acc: 0.8651
Epoch 2/5
180/180 [==============================] - 2140s - loss: 0.6521 - acc: 0.8745 - val_loss: 0.5965 - val_acc: 0.8857
Epoch 3/5
180/180 [==============================] - 2266s - loss: 0.5268 - acc: 0.8967 - val_loss: 0.4909 - val_acc: 0.9056
Epoch 4/5
180/180 [==============================] - 2257s - loss: 0.4398 - acc: 0.9117 - val_loss: 0.4255 - val_acc: 0.9170
Epoch 5/5
180/180 [==============================] - 2076s - loss: 0.3856 - acc: 0.9202 - val_loss: 0.3832 - val_acc: 0.9228
